In [2]:
import pandas as pd 
from elasticsearch6 import Elasticsearch
import json

In [4]:
index_cluster = "events-20220502",
path_json_query = "/home/eguimard/PycharmProjects/DataStudy/DataStorage/JsonQuery/eventTraceabilityVM1.json"

In [5]:
def load_json_file(path_file):
    with open(path_file) as file:
        json_file = json.load(file)
        file.close()
        return json_file


class ElasticAPI:

    def __init__(self, host='localhost', port=9200):
        try:
            self.client = Elasticsearch(f'{host}:{port}')
        except ConnectionError:  # check the except
            self.client = None
        else:
            print('Connected')
    
    
    def INDEV_get_raw_data_from_json_query(self, index, path_json_query):
        str_json_query = load_json_file(path_json_query)
        return self.client.search(index=index, body=str_json_query)

In [6]:
raw_data = ElasticAPI().INDEV_get_raw_data_from_json_query(index_cluster, path_json_query)
raw_data

Connected


{'took': 97,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': 812291,
  'max_score': 1.6984568,
  'hits': [{'_index': 'events-20220502',
    '_type': '_doc',
    '_id': '9QWVkoABdRhNzRXUqF3b',
    '_score': 1.6984568,
    '_source': {'category': 'traceability',
     'summary': 'Host ping',
     'client_id': 'airliquide',
     'hostname': 'VM1',
     'timestamp': '2022-05-05T04:58:20.000Z',
     'source': 'heartbeat',
     'tags': ['heartbeat.uptime'],
     'details': {'event': {'dataset': 'uptime'},
      'hostname': 'test-conformite.netplus.priv',
      'url': {'scheme': 'icmp',
       'domain': '192.168.122.3',
       'full': 'icmp://192.168.122.3'},
      'summary': {'down': 0, 'up': 1},
      'ecs': {'version': '1.5.0'},
      'agent': {'id': '016bd43b-ef2a-47c2-8e55-e991357aec94',
       'type': 'heartbeat',
       'version': '7.9.3',
       'hostname': 'test-conformite.netplus.priv',
       'name': 'test-conformite.netp

In [7]:
raw_data['hits']['hits'][0]['_source']['details'].keys()

dict_keys(['event', 'hostname', 'url', 'summary', 'ecs', 'agent', 'icmp', 'monitor', 'endpoint', 'customendpoint'])

In [6]:
class PrepareData:

    def _transform_raw_data_into_dataframe(self, raw_data):
        # !! need to improve the treatment of bad query

        # get the return of the query
        list_results = raw_data['hits']['hits']
        # create df and columns
        try:
            df = pd.DataFrame(columns=list_results[0]['_source'].keys())  # check le 0
        except IndexError as err:
            # error when result[0] doesn't exist
            print(f'ERROR in the query -> no existent raw_data_source')
            return pd.DataFrame()

        # append results as row
        for result in list_results:
            df.loc[df.shape[0]] = list(result['_source'].values())

        return df
    
    
    def get_relevant_data(self, raw_data):
        
        
        dict_dataframe = dict()
        
        list_results = raw_data['hits']['hits']
        
        for results in list_results:
            
            system_data = results['_source']['details']['system']
            
            perimeter_impacted = list(system_data.keys())[0]
            
            if perimeter_impacted not in dict_dataframe:
                dict_dataframe[perimeter_impacted] = pd.DataFrame(columns=system_data[perimeter_impacted].keys())
                
              
            next_line = dict_dataframe[perimeter_impacted].shape[0]
            dict_dataframe[perimeter_impacted].loc[next_line] = list(system_data[perimeter_impacted].values()) #[perimeter_impacted]
        
        return dict_dataframe
            
            
           #df.loc[df.shape[0]] = list(result['_source'].values())
        
        

In [111]:
class PrepareData:
    
    
    
    def extract_relevant_data_in_dict(self, raw_data):

        raw_data_in_dict = dict()

        list_results = raw_data['hits']['hits']

        for results in list_results:

            data_system = results['_source']['details']['system']

            system_name = list(data_system.keys())[0]
        
            if system_name not in raw_data_in_dict:
                raw_data_in_dict[system_name] = list()
            
            
            raw_data_in_dict[system_name].append(self.recursive_sort_data(data_system[system_name]))
                  
        return raw_data_in_dict
    
    
    def recursive_sort_data(self, current_object, name_feature=''):
        dict_result = dict()

        if type(current_object) is dict:
            for key, new_obj in current_object.items():
                dict_result.update(self.recursive_sort_data(new_obj, f'{name_feature}{key}_'))
        else:
            dict_result[name_feature[:-1]] = current_object
            # list_result.append((name_feature[:-1], current_object))

        return dict_result

In [115]:
pd.DataFrame(PrepareData().extract_relevant_data_in_dict(raw_data)['memory'])

,total,used_pct,used_bytes,actual_free,actual_used_pct,actual_used_bytes,page_stats_pgsteal_direct_pages,page_stats_pgsteal_kswapd_pages,page_stats_direct_efficiency_pct,page_stats_kswapd_efficiency_pct,...,free,hugepages_swap_out_pages,hugepages_swap_out_fallback,hugepages_total,hugepages_used_pct,hugepages_used_bytes,hugepages_default_size,hugepages_free,hugepages_surplus,hugepages_reserved
0,1028845568,0.9126,938954752,611532800,0.4056,417312768,125,552583,1,0.8723,...,89890816,0,0,0,0,0,2097152,0,0,0
1,1028845568,0.9126,938954752,611536896,0.4056,417308672,125,552583,1,0.8723,...,89890816,0,0,0,0,0,2097152,0,0,0
2,1028845568,0.9126,938954752,611627008,0.4055,417218560,125,552583,1,0.8723,...,89890816,0,0,0,0,0,2097152,0,0,0
3,1028845568,0.9126,938921984,611659776,0.4055,417185792,125,552583,1,0.8723,...,89923584,0,0,0,0,0,2097152,0,0,0
4,1028845568,0.9126,938921984,611659776,0.4055,417185792,125,552583,1,0.8723,...,89923584,0,0,0,0,0,2097152,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,1028845568,0.9175,943955968,610951168,0.4062,417894400,125,552583,1,0.8723,...,84889600,0,0,0,0,0,2097152,0,0,0
247,1028845568,0.9175,943955968,610934784,0.4062,417910784,125,552583,1,0.8723,...,84889600,0,0,0,0,0,2097152,0,0,0
248,1028845568,0.9175,943955968,610942976,0.4062,417902592,125,552583,1,0.8723,...,84889600,0,0,0,0,0,2097152,0,0,0
249,1028845568,0.9170,943439872,611487744,0.4057,417357824,125,552583,1,0.8723,...,85405696,0,0,0,0,0,2097152,0,0,0


In [7]:
l = PrepareData()._transform_raw_data_into_dataframe(raw_data)

In [8]:
ddf = PrepareData().get_relevant_data(raw_data)

In [11]:
list(ddf.keys())

['memory', 'fsstat', 'cpu', 'network']

In [105]:
for row in range(ddf['memory'].shape[0]):
    print(dict(ddf['memory'].loc[row]))

{'total': 1028845568, 'used': {'pct': 0.9126, 'bytes': 938954752}, 'actual': {'free': 611532800, 'used': {'pct': 0.4056, 'bytes': 417312768}}, 'page_stats': {'pgsteal_direct': {'pages': 125}, 'pgsteal_kswapd': {'pages': 552583}, 'direct_efficiency': {'pct': 1}, 'kswapd_efficiency': {'pct': 0.8723}, 'pgfree': {'pages': 18497109}, 'pgscan_kswapd': {'pages': 633473}, 'pgscan_direct': {'pages': 125}}, 'swap': {'in': {'pages': 0}, 'readahead': {'pages': 0, 'cached': 0}, 'used': {'pct': 0, 'bytes': 0}, 'total': 0, 'free': 0, 'out': {'pages': 0}}, 'free': 89890816, 'hugepages': {'swap': {'out': {'pages': 0, 'fallback': 0}}, 'total': 0, 'used': {'pct': 0, 'bytes': 0}, 'default_size': 2097152, 'free': 0, 'surplus': 0, 'reserved': 0}}
{'total': 1028845568, 'used': {'pct': 0.9126, 'bytes': 938954752}, 'actual': {'free': 611536896, 'used': {'pct': 0.4056, 'bytes': 417308672}}, 'page_stats': {'pgsteal_direct': {'pages': 125}, 'pgsteal_kswapd': {'pages': 552583}, 'direct_efficiency': {'pct': 1}, 'ks

In [107]:
 print(dict(ddf['memory'].loc[0]))

{'total': 1028845568, 'used': {'pct': 0.9126, 'bytes': 938954752}, 'actual': {'free': 611532800, 'used': {'pct': 0.4056, 'bytes': 417312768}}, 'page_stats': {'pgsteal_direct': {'pages': 125}, 'pgsteal_kswapd': {'pages': 552583}, 'direct_efficiency': {'pct': 1}, 'kswapd_efficiency': {'pct': 0.8723}, 'pgfree': {'pages': 18497109}, 'pgscan_kswapd': {'pages': 633473}, 'pgscan_direct': {'pages': 125}}, 'swap': {'in': {'pages': 0}, 'readahead': {'pages': 0, 'cached': 0}, 'used': {'pct': 0, 'bytes': 0}, 'total': 0, 'free': 0, 'out': {'pages': 0}}, 'free': 89890816, 'hugepages': {'swap': {'out': {'pages': 0, 'fallback': 0}}, 'total': 0, 'used': {'pct': 0, 'bytes': 0}, 'default_size': 2097152, 'free': 0, 'surplus': 0, 'reserved': 0}}


In [70]:
dict(ddf['memory'].loc[0])

{'total': 1028845568,
 'used': {'pct': 0.9126, 'bytes': 938954752},
 'actual': {'free': 611532800, 'used': {'pct': 0.4056, 'bytes': 417312768}},
 'page_stats': {'pgsteal_direct': {'pages': 125},
  'pgsteal_kswapd': {'pages': 552583},
  'direct_efficiency': {'pct': 1},
  'kswapd_efficiency': {'pct': 0.8723},
  'pgfree': {'pages': 18497109},
  'pgscan_kswapd': {'pages': 633473},
  'pgscan_direct': {'pages': 125}},
 'swap': {'in': {'pages': 0},
  'readahead': {'pages': 0, 'cached': 0},
  'used': {'pct': 0, 'bytes': 0},
  'total': 0,
  'free': 0,
  'out': {'pages': 0}},
 'free': 89890816,
 'hugepages': {'swap': {'out': {'pages': 0, 'fallback': 0}},
  'total': 0,
  'used': {'pct': 0, 'bytes': 0},
  'default_size': 2097152,
  'free': 0,
  'surplus': 0,
  'reserved': 0}}

In [13]:
field_total = ddf['memory'].loc[0]['total']
field_used = ddf['memory'].loc[0]['used']
field_actual = ddf['memory'].loc[0]['actual']
field_page_stats = ddf['memory'].loc[0]['page_stats']
field_swap = ddf['memory'].loc[0]['swap']
field_free = ddf['memory'].loc[0]['free']
field_hugepages = ddf['memory'].loc[0]['hugepages']

In [68]:
field_total

1028845568

In [14]:
field_used

{'pct': 0.9126, 'bytes': 938954752}

In [15]:
field_actual

{'free': 611532800, 'used': {'pct': 0.4056, 'bytes': 417312768}}

In [16]:
field_page_stats

{'pgsteal_direct': {'pages': 125},
 'pgsteal_kswapd': {'pages': 552583},
 'direct_efficiency': {'pct': 1},
 'kswapd_efficiency': {'pct': 0.8723},
 'pgfree': {'pages': 18497109},
 'pgscan_kswapd': {'pages': 633473},
 'pgscan_direct': {'pages': 125}}

In [17]:
field_swap

{'in': {'pages': 0},
 'readahead': {'pages': 0, 'cached': 0},
 'used': {'pct': 0, 'bytes': 0},
 'total': 0,
 'free': 0,
 'out': {'pages': 0}}

In [18]:
field_free

89890816

In [19]:
field_hugepages

{'swap': {'out': {'pages': 0, 'fallback': 0}},
 'total': 0,
 'used': {'pct': 0, 'bytes': 0},
 'default_size': 2097152,
 'free': 0,
 'surplus': 0,
 'reserved': 0}

In [20]:
new_df = pd.DataFrame()

In [93]:
class test:
    
    def sort_data(self, dict_of_dict, name_feature):

        sorted_dict = self.recursive_sort_data(dict_of_dict, name_feature)
        return sorted_dict

    def recursive_sort_data(self, current_object, name_feature=''):
        dict_result = dict()

        if type(current_object) is dict:
            for key, new_obj in current_object.items():
                dict_result.update(self.recursive_sort_data(new_obj,f'{name_feature}{key}_'))
        else:
            dict_result[name_feature[:-1]] = current_object
            #list_result.append((name_feature[:-1], current_object))

        return dict_result
    
        

In [97]:
field_page_stats

{'pgsteal_direct': {'pages': 125},
 'pgsteal_kswapd': {'pages': 552583},
 'direct_efficiency': {'pct': 1},
 'kswapd_efficiency': {'pct': 0.8723},
 'pgfree': {'pages': 18497109},
 'pgscan_kswapd': {'pages': 633473},
 'pgscan_direct': {'pages': 125}}

In [83]:
name_feature ='rjerui9'
name_feature[:-1]

'rjerui'

In [94]:
a = test().sort_data(dict(ddf['memory'].loc[0]),'')
a

{'total': 1028845568,
 'used_pct': 0.9126,
 'used_bytes': 938954752,
 'actual_free': 611532800,
 'actual_used_pct': 0.4056,
 'actual_used_bytes': 417312768,
 'page_stats_pgsteal_direct_pages': 125,
 'page_stats_pgsteal_kswapd_pages': 552583,
 'page_stats_direct_efficiency_pct': 1,
 'page_stats_kswapd_efficiency_pct': 0.8723,
 'page_stats_pgfree_pages': 18497109,
 'page_stats_pgscan_kswapd_pages': 633473,
 'page_stats_pgscan_direct_pages': 125,
 'swap_in_pages': 0,
 'swap_readahead_pages': 0,
 'swap_readahead_cached': 0,
 'swap_used_pct': 0,
 'swap_used_bytes': 0,
 'swap_total': 0,
 'swap_free': 0,
 'swap_out_pages': 0,
 'free': 89890816,
 'hugepages_swap_out_pages': 0,
 'hugepages_swap_out_fallback': 0,
 'hugepages_total': 0,
 'hugepages_used_pct': 0,
 'hugepages_used_bytes': 0,
 'hugepages_default_size': 2097152,
 'hugepages_free': 0,
 'hugepages_surplus': 0,
 'hugepages_reserved': 0}

In [ ]:
ddf['fsstat']

In [ ]:
ddf['cpu']

In [ ]:
ddf['network']

In [ ]:
list_results = raw_data['hits']['hits'][0]['_source']['details'][0]
list_results
#list(results['_source']['details'][0]['system'].keys())[0]

In [ ]:
df = pd.DataFrame()
for result in l:
    df.loc[df.shape[0]] = list(result['_source'].values())

In [ ]:
raw_data['hits']['hits'][1]['_source']['details']['system']['fsstat'].keys()